In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/16 10:07:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import zipfile
import io


def zip_extract(x):
    in_memory_data = io.BytesIO(x[1])
    file_obj = zipfile.ZipFile(in_memory_data, "r")
    files = [i for i in file_obj.namelist()]
    return [(str(file_obj.open(file).read().decode("UTF-8")),) for file in files]

In [4]:
zips = spark.sparkContext.binaryFiles("hdfs://127.0.0.1:9000/fedora/raw/stream/*")

In [5]:
files_data = zips.flatMap(zip_extract).toDF()

In [6]:
from pyspark.sql.types import *

schema = StructType([
    StructField("year", StringType()),
    StructField("city", StringType()),
    StructField("price", StringType()),
    StructField("vin", StringType()),
    StructField("model", StringType()),
    StructField("state", StringType()),
    StructField("make", StringType()),
    StructField("mileage", StringType())
])

In [7]:
from pyspark.sql.functions import *

In [8]:
df = files_data.withColumn("jsonData", from_json(col("_1"), schema)).select("jsonData.*")

In [9]:
df = df.withColumn("year", df.year.cast("int"))\
    .withColumn("price", df.price.cast("int"))\
    .withColumn("mileage", df.mileage.cast("int"))

In [11]:
df.toPandas().head()

,year,city,price,vin,model,state,make,mileage
0,2010,Lemont,15990,WDCGG8HB3AF286045,GLK-Class4MATIC,IL,Mercedes-Benz,99042
1,2014,Hillsboro,23950,WDDGF8AB8EA942297,C-ClassC300,OR,Mercedes-Benz,39888
2,2014,Doraville,22599,WDDSJ4GB8EN121640,CLA-ClassCLA250,GA,Mercedes-Benz,50313
3,2015,Walnut Creek,25500,55SWF4JB3FU059956,C-ClassC300,CA,Mercedes-Benz,29825
4,2014,Columbus,25950,WDDSJ4GB6EN105789,CLA-ClassCLA250,OH,Mercedes-Benz,21468
